<a href="https://colab.research.google.com/github/kookeej/DILAB/blob/main/7.29-8.11/document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

문서 분류 (Document Classification)
===
> *reference: https://www.youtube.com/watch?v=xegxbgsnYko&list=PL7ZVZgsnLwEEoHQAElEPg7l7T6nt25I3N&index=4*
# 데이터 준비
* 문서 분류에 필요한 데이터는 ```sklearn```이 제공하는 20개의 주제를 가지는 뉴스 그룹을 사용한다.
* 텍스트는 `CountVectorizer`를 거쳐 DTM으로 변환한다.
* DTM은 문ㅅ에 등장하는 단어들을 빈도수별로 표현한 행렬이다.

In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

news = fetch_20newsgroups()

x = news.data
y = news.target

cv = CountVectorizer()
x = cv.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(7919, 130107) (7919,) (3395, 130107) (3395,)


In [2]:
# (0, 56979) 1 -> 0이라는 문서에 56979 인덱스를 갖는 단어가 1번 등장.
print(x_train[0])

  (0, 56979)	1
  (0, 85354)	1
  (0, 111322)	1
  (0, 123984)	1
  (0, 68532)	6
  (0, 114731)	3
  (0, 89362)	3
  (0, 76032)	1
  (0, 114579)	1
  (0, 80638)	3
  (0, 114455)	4
  (0, 68766)	3
  (0, 115475)	12
  (0, 32311)	1
  (0, 37433)	1
  (0, 99822)	1
  (0, 66608)	6
  (0, 27436)	1
  (0, 90252)	3
  (0, 128402)	2
  (0, 35983)	1
  (0, 37423)	1
  (0, 56283)	1
  (0, 114494)	1
  (0, 81263)	2
  :	:
  (0, 39072)	2
  (0, 54786)	1
  (0, 46355)	4
  (0, 111149)	3
  (0, 53951)	2
  (0, 85780)	2
  (0, 55212)	2
  (0, 96394)	1
  (0, 52915)	1
  (0, 99238)	1
  (0, 96385)	1
  (0, 96386)	2
  (0, 78602)	1
  (0, 98332)	1
  (0, 27913)	1
  (0, 107491)	1
  (0, 25894)	1
  (0, 60456)	1
  (0, 39203)	1
  (0, 115138)	1
  (0, 51142)	1
  (0, 38170)	1
  (0, 3113)	1
  (0, 10250)	1
  (0, 120286)	1


# scikit-learn을 이용한 문서 분류


In [3]:
from sklearn.metrics import accuracy_score

## Logistic Regression
* `logistic regression`은 이름에 회귀라는 단어가 들어가지만 클래스의 개수가 2개인 이진 분류를 위한 모델이다.
* logistic regression은 linear regression model에 시그모이드 함수를 적용한 것이다.
* logistic regression의 학습 목표는 목적 함수인 `BinaryCrossEntropy`를 최소화하는 파라미터 w를 찾는 것이다.
* 다중 분류에는 적합하지 않다.

In [4]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(x_train, y_train)
pred = LR.predict(x_test)
acc = accuracy_score(pred, y_test)      # 예측값과 실제값을 비교하여 정확도를 구한다.
print(acc)

0.8739322533136966


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


생각보다 높은 성능이 나왔지만 다중 분류에는 적합하지 않는 모델이기 때문에 logistic regression을 사용하기에는 어려움이 있다.

## 서포트 벡터 머신(Support Vector Machines)
* 회귀(regression), 분류(classification), 이상치 탐지(outlier detection)에 사용된는 지도학습 방법이다.
* 클래스 사이의 경계에 위치한 데이터 포인트를 `support vector`라고 한다. 
* 각 서포트 벡터가 클래스 사이의 `결정 경계`를 구분하는데 얼마나 중요한지를 학습한다.
* 각 서포트 벡터 사이의 마진이 가장 큰 방향으로 학습한다. 데이터와 데이터 사이의 거리가 최대가 되도록 학습한다.
* 서포트 벡터까지의 **거리**와 서포트 벡터의 **중요도**를 기반으로 예측을 수행한다.

In [5]:
from sklearn import svm

SVM = svm.SVC(kernel='linear')
SVM.fit(x_train, y_train)
pred = SVM.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.8285714285714286


## 나이브 베이스 분류기(Naive Bayes Classification)
* 베이즈 정리를 적용한 확률적 분류 알고리즘이다.
* 모든 feature들이 독립임을 가정한다. (naive 가정)
* 입력 특성에 따라 3개의 분류기가 존재한다.
    * 가우시안 나이브 베이즈 분류기
    * 베르누이 나이브 베이즈 분류기
    * 다항 나이브 베이즈 분류기(DTM 이용)


> ### DTM을 이용한 나이브 베이즈

In [6]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()
NB.fit(x_train, y_train)
pred = NB.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.8206185567010309


> ### TF-IDF를 이용한 정확도 향상


In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
x_train_tf = tfidf.fit_transform(x_train)
x_test_tf = tfidf.fit_transform(x_test)

NB.fit(x_train_tf, y_train)
pred = NB.predict(x_test_tf)
acc = accuracy_score(pred, y_test)
print(acc)

0.81620029455081


엥 향상 안됨

## 결정 트리(Decision Tree)
* 분류(classification)와 회귀(regression)에 사용되는 지도 학습 방법이다.
* 데이터 feature로부터 추론된 결정 규칙을 통해 값을 예측한다.
* **if-then-else** 결정 규칙을 통해 데이터를 학습한다.
* 트리의 깊이가 깊을수록 복잡한 모델이고 오버피팅이 발생할 위험이 크다.

In [10]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(x_train, y_train)
pred = DT.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.6329896907216495


현재 아무런 처리도 하지 않은 데이터로써는 높은 성능을 기대하기 어렵다. 상당히 좋지 않은 결과가 나왔다.

## XGBoot
* 트리 기반의 앙상블 기법이다.
* 분류에 있어서 다른 알고리즘보다 좋은 예측 성능을 보여준다.
* XGBoost는 GBM 기반이지만 GBM의 단점인 느린 수행 시간과 과적합 규제 부재 등의 문제를 해결하였다.
* 병렬 CPU 환경에서 빠르게 학습 가능하다.

In [13]:
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=30, learning_rate=0.05, max_depth=3)
xgb.fit(x_train, y_train)
pred = xgb.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.7154639175257732


단순 Decision Tree보다는 조금 더 좋은 성능이 나왔다.

## 교차 검증 (Cross Validation)
* 일반 검증은 학습 데이터가 테스트 데이터로 사용되지 않는다.
* 교차 검증은 데이터를 n개의 집합으로 나누어 정확도를 계산해 학습 데이터로 사용된 데이터도 테스트 데이터로 사용한다.
* 교차 검증을 사용하면 일반 검증보다 **모델의 일반화가 잘 되어있는지** 평가 가능하다.    

앞에서 학습시킨 나이브 베이즈 모델을 교차 검증 해보자.

In [14]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(NB, x, y, cv=5)       # 5개의 집합으로 구분
print(scores, scores.mean())

[0.83870968 0.83826779 0.82368537 0.83031374 0.83642794] 0.833480903927519


교차검증을 통해 일반 검증보다 좀 더 일반화된 모델을 생성할 수 있다. 하지만 교차 검증은 일반 검증에 비해 n번 검증을 더 하기 때문에 비용이 더 크다.

## 정밀도(precision)와 재현율(recall)
* `정밀도(precision)`는 양성 클래스(정답)으로 예측한 샘플이 양성 클래스일 확률을 의미한다. 즉, 모델이 얼마나 양성 클래스를 잘 예측하는지를 나타낸다.
* `재현율(recall)`은 양성 클래스(정답)인 샘플에서 모델이 양성 클래스로 예측한 샘플 비율을 의미하며, 모델이 얼마나 실제 상황을 재현하는지를 나타낸다.
* 정밀도와 재현율의 **가중조화평균인 F1-score**라는 지표는 정확도에 비해 더 효과적인 모델 분석 지표로 알려져 있다.      

직접 계산할 수도 있으나 `sklearn`은 이를 편리하게 계산해주는 함수를 제공한다.    

* 다중 클래스 분류 문제에서 정밀도와 재현율을 계산할 때는 클래스간의 지표를 어떻게 합칠지 정해야 한다.
    * `None`: 클래스간 지표를 합치지 말고 그대로 출력
    * `micro`: 정밀도와 재현율이 같음. 이로 인해 f1-score도 정밀도, 재현율과 동일
    * `macro`: 클래스간 지표를 단순 평균화한 값
    * `weighted`: 클래스간 지표를 가중 평균화한 값



In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(pred, y_test, average='micro')
recall = recall_score(pred, y_test, average='micro')
f1 = f1_score(pred, y_test, average='micro')

print(precision, recall, f1)

0.7154639175257732 0.7154639175257732 0.7154639175257732


micro는 정밀도, 재현율, f1-score 보두 같은 값으로 나온다.

In [17]:
precision = precision_score(pred, y_test, average='macro')
recall = recall_score(pred, y_test, average='macro')
f1 = f1_score(pred, y_test, average='macro')

print(precision, recall, f1)

0.7116528251239695 0.7449139649401825 0.7215351690399557


# GridSearchCV를 통한 파라미터 최적화
`GridSearchCV`는 `Grid Search + Cross Validation`을 뜻한다. 이것을 사용하면 분류기에 사용하는 파라미터 최적화가 가능하다. 또한 교차 검증까지 함께 할 수 있다. 그리그 검색을 통해 앞서 구성한 나이브 베이즈 모델의 'alpha' 파라미터를 최적화시켜보자.

* `estimator`: 사용 모델 객체
* `param_grid`: 사용 객체. 지정 파라미터 리스트로 구성된 딕셔너리
* `scoring`: 최적화하고자 하는 성능 지표
* `cv`: 교차 검증 분할 개수

In [19]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(estimator=NB, param_grid={'alpha': [0.001, 0.01, 0.1, 1.]}, scoring='accuracy', cv=10)
gs.fit(x, y)

print(gs.best_score_)
print(gs.best_params_)

0.8897820965842167
{'alpha': 0.001}


성능이 88%까지 증가한 것을 확인할 수 있고 alpha값은 0.001이 최적이 파라미터인 것을 확인할 수 있다.